## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saskylakh,RU,71.9167,114.0833,-24.77,98,11,8.68,few clouds
1,1,Chokurdakh,RU,70.6333,147.9167,-12.50,99,100,5.48,overcast clouds
2,2,Esperance,AU,-33.8667,121.9000,63.18,67,83,16.91,broken clouds
3,3,Albany,US,42.6001,-73.9662,32.36,71,20,1.99,few clouds
4,4,Vaini,TO,-21.2000,-175.2000,80.76,89,40,6.91,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Vaini,TO,-21.2000,-175.2000,80.76,89,40,6.91,scattered clouds
8,8,Atuona,PF,-9.8000,-139.0333,77.74,75,100,9.10,overcast clouds
9,9,Dingle,PH,10.9995,122.6711,85.82,72,67,13.20,broken clouds
11,11,Bengkulu,ID,-3.8004,102.2655,83.35,65,90,4.45,overcast clouds
13,13,Butaritari,KI,3.0707,172.7902,82.26,77,20,11.61,few clouds
18,18,Rikitea,PF,-23.1203,-134.9692,75.90,76,1,11.36,clear sky
21,21,Langsa,ID,4.4683,97.9683,87.10,64,100,2.13,light rain
22,22,Hithadhoo,MV,-0.6000,73.0833,81.12,79,100,17.96,light rain
23,23,Carnarvon,AU,-24.8667,113.6333,77.07,53,0,21.85,clear sky
24,24,Hilo,US,19.7297,-155.0900,75.27,96,90,0.00,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                193
City                   193
Country                193
Lat                    193
Lng                    193
Max Temp               193
Humidity               193
Cloudiness             193
Wind Speed             193
Current Description    193
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,
8,Atuona,PF,77.74,overcast clouds,-9.8000,-139.0333,
9,Dingle,PH,85.82,broken clouds,10.9995,122.6711,
11,Bengkulu,ID,83.35,overcast clouds,-3.8004,102.2655,
13,Butaritari,KI,82.26,few clouds,3.0707,172.7902,
18,Rikitea,PF,75.90,clear sky,-23.1203,-134.9692,
21,Langsa,ID,87.10,light rain,4.4683,97.9683,
22,Hithadhoo,MV,81.12,light rain,-0.6000,73.0833,
23,Carnarvon,AU,77.07,clear sky,-24.8667,113.6333,
24,Hilo,US,75.27,moderate rain,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace('', float("NaN"), inplace = True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   180
Country                180
Max Temp               180
Current Description    180
Lat                    180
Lng                    180
Hotel Name             180
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))